In [ ]:
from fastai.vision.all import *

In [ ]:
path = Path("/kaggle/input/pascal-2007/pascal_2007")

In [ ]:
df = pd.read_csv(path/'train.csv')
df.head()

In [ ]:
df.iloc[:,0]

In [ ]:
df.iloc[0,:]

In [ ]:
df.iloc[0]

In [ ]:
df['fname']

In [ ]:
df1=pd.DataFrame()
df1['a'] = [1,2,3,4]
df1

In [ ]:
df1['b'] = [10,20,30,40]
df1,df1['a']+df1['b']

In [ ]:
dblock = DataBlock()

In [ ]:
dsets = dblock.datasets(df)

In [ ]:
dsets.train[0]

In [ ]:
dblock1 = DataBlock(get_x = lambda r: r['fname'],get_y = lambda r: r['labels'])
dsets1 = dblock1.datasets(df)
dsets1.train[0]

In [ ]:
# equal function codes:
# def get_x(r):return r['fname']
def get_x(r):return path/'train'/r['fname']
def get_y(r):return r['labels'].split(' ')
dblock = DataBlock(get_x = get_x,get_y = get_y)
dsets = dblock.datasets(df)
dsets.train[0]

In [ ]:
dblock = DataBlock(blocks=(ImageBlock,MultiCategoryBlock),
                   get_x = get_x,get_y = get_y)
dsets = dblock.datasets(df)
dsets.train[0]

In [ ]:
idxs = torch.where(dsets.train[0][1]==1.)[0]
dsets.train.vocab[idxs]

In [ ]:
idxs

In [ ]:
dsets.train.vocab

In [ ]:
def splitter(df):
    train = df.index[~df['is_valid']].tolist()
    valid = df.index[df['is_valid']].tolist()
    return train,valid

In [ ]:
dblock = DataBlock(blocks=(ImageBlock,MultiCategoryBlock),
                   splitter = splitter,
                   get_x = get_x,get_y = get_y)
dsets = dblock.datasets(df)
dsets.train[0]

In [ ]:
dblock = DataBlock(blocks=(ImageBlock,MultiCategoryBlock),
                   splitter = splitter,
                   get_x = get_x,get_y = get_y,
                  item_tfms = RandomResizedCrop(128,min_scale = 0.35))
dls = dblock.dataloaders(df)

In [ ]:
dls.show_batch(nrows=1,ncols=3)

In [ ]:
lesrn = vision_learner(dls,resnet18)

In [ ]:
x,y = dls.train.one_batch()
activs = learn.model(x)
activs.shape

In [ ]:
activs[0]

In [ ]:
def binary_cross_entropy(inputs,targets):
    inputs = inputs.sigmoid()
    return -torch.where(targets==1,inputs,1-inputs).log().mean()